In [2]:
import bs4
import requests
import pandas as pd

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) '
                    'AppleWebKit/537.36 (KHTML, like Gecko) '
                    'Chrome/92.0.4515.107 Safari/537.36',
}

In [3]:
fixtures = pd.read_csv("https://raw.githubusercontent.com/petebrown/update-results/main/data/results_df.csv")
games = fixtures[["season", "game_date"]].to_dict(orient="records")

year = games[0]["season"].split("/")[0]
game_date = games[0]["game_date"]

### Yellow and Red Cards

In [ ]:
records = []
for game in games:
    year = game["season"].split('/')[0]
    if int(year) < 1978:
        next
    else:
        game_date = game["game_date"]

        url = f"https://www.tranmere-web.com/match/{year}/{game_date}"
        print(url)

        try:
            r = requests.get(url, headers=headers)

            soup = bs4.BeautifulSoup(r.text, 'lxml')
            table = soup.find("table", {"class" :"pitch"})

            if table:
                colors = ["yellow", "red"]

                for color in colors:
                    all_cards = table.find_all("a", {"class": color})
                    if all_cards:
                        for card in all_cards:
                            record = {
                                "game_date": game_date,
                                "player": card.text,
                                "card": color,
                                "url": url   
                            }
                            records.append(record)
                            print(record)
                    else:
                        print(f"No {color} cards in {url}")
            else:
                print(f"No table on: {url}")

        except Exception as e:
            print(url, e)

df = pd.DataFrame(records)
df.to_csv('tranmere-web-cards.csv', index=False)

### URLs of programme cover images

In [10]:
programmes = []
for game in games:
    game_date = game["game_date"]

    year = game["season"].split('/')[0]

    url = f"https://www.tranmere-web.com/match/{year}/{game_date}"

    try:
        r = requests.get(url, headers=headers)

        soup = bs4.BeautifulSoup(r.text, 'lxml')
        prog = soup.find("div", {"class" :"widget"}).find("img")

        if prog:
            img_url = prog["src"]
            record = {
                "game_date": game_date,
                "programme": img_url,
                "url": url   
            }
            programmes.append(record)
            print(record)
        else:
            print(f"No programme in {url}")

    except Exception as e:
        print(url, e)

progs_df = pd.DataFrame(programmes)
progs_df.head()

No programme in https://www.tranmere-web.com/match/2022/2023-05-08
No programme in https://www.tranmere-web.com/match/2022/2023-04-29
No programme in https://www.tranmere-web.com/match/2022/2023-04-22
No programme in https://www.tranmere-web.com/match/2022/2023-04-18
No programme in https://www.tranmere-web.com/match/2022/2023-04-15
No programme in https://www.tranmere-web.com/match/2022/2023-04-10
No programme in https://www.tranmere-web.com/match/2022/2023-04-07
No programme in https://www.tranmere-web.com/match/2022/2023-03-31
No programme in https://www.tranmere-web.com/match/2022/2023-03-25
No programme in https://www.tranmere-web.com/match/2022/2023-03-18
No programme in https://www.tranmere-web.com/match/2022/2023-03-11
No programme in https://www.tranmere-web.com/match/2022/2023-03-04
No programme in https://www.tranmere-web.com/match/2022/2023-02-28
No programme in https://www.tranmere-web.com/match/2022/2023-02-25
No programme in https://www.tranmere-web.com/match/2022/2023-0

,game_date,programme,url
0,2022-04-18,https://images.tranmere-web.com/eyJidWNrZXQiOi...,https://www.tranmere-web.com/match/2021/2022-0...
1,2022-03-15,https://images.tranmere-web.com/eyJidWNrZXQiOi...,https://www.tranmere-web.com/match/2021/2022-0...
2,2022-02-19,https://images.tranmere-web.com/eyJidWNrZXQiOi...,https://www.tranmere-web.com/match/2021/2022-0...
3,2022-01-29,https://images.tranmere-web.com/eyJidWNrZXQiOi...,https://www.tranmere-web.com/match/2021/2022-0...
4,2022-01-15,https://images.tranmere-web.com/eyJidWNrZXQiOi...,https://www.tranmere-web.com/match/2021/2022-0...


### Download program cover images

In [ ]:
import requests

progs_dic = progs_df.to_dict('records')

for prog in progs_dic:
    filename = prog["game_date"]
    url = prog["programme"]
    
    r = requests.get(url, allow_redirects = True)
    open(f'./progs/{filename}.jpg', 'wb').write(r.content)
    print(f"Added {filename}.jpg")